# Extract all tables in `sales` schema

In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import json
from datetime import datetime, timedelta
from pyspark.sql.functions import lit


In [64]:
with open('config.json', 'r') as f:
    config = json.load(f)

## Control variables

In [65]:
schema = config["general"]["schema"]
catalog = f'{config["general"]["user"]}_raw'
db_password = dbutils.secrets.get(scope="antonio_junior_adw", key="pswd_mssql")
db_host = dbutils.secrets.get(scope="antonio_junior_adw", key="ip_mssql")
db_port = dbutils.secrets.get(scope="antonio_junior_adw", key="port_mssql")
db_user = config["general"]["db_user"]
database =  config["general"]["database"]
tables_list = list(config["tables"].keys())
last_modified_date = datetime.today() - timedelta(days=7)
last_modified_date = last_modified_date.strftime('%Y-%m-%d 00:00:00')
last_modified_date = "2007-01-01 00:00:00"

Connections vars

In [66]:
jdbc_url = f"jdbc:sqlserver://{db_host}:{db_port};databaseName={database};encrypt=true;trustServerCertificate=true;"

connection_properties = {
    "user": db_user, 
    "password": db_password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# SQL QUERIES

List tables in schema

Create schema

In [68]:
create_schema = f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}"
spark.sql(create_schema)


""


## Extraction steps

Extract all data from each table.

In [69]:
for table_name in tables_list:
    delta_table_name = f"{catalog}.{schema}.{table_name}"
    print(table_name)

    query = f"(SELECT * FROM {schema}.{table_name} WHERE ModifiedDate >= '{last_modified_date}') AS subquery"

    incremental_df = spark.read.jdbc(
        url=jdbc_url,
        table=query,
        properties=connection_properties
    )

    incremental_df = incremental_df.withColumn("extract_date", lit(datetime.today()))

    incremental_df.write.mode("overwrite").format("delta").saveAsTable(delta_table_name)

CountryRegionCurrency
CreditCard
Currency
CurrencyRate
Customer
PersonCreditCard
SalesOrderDetail
SalesOrderHeader
SalesOrderHeaderSalesReason
SalesPerson
SalesPersonQuotaHistory
SalesReason
SalesTaxRate
SalesTerritory
SalesTerritoryHistory
ShoppingCartItem
SpecialOffer
SpecialOfferProduct
Store
